In [1]:
import numpy as np
from skimage import io

import os
import napari
import pickle

# from sklearn.preprocessing import normalize, PolynomialFeatures
# from sklearn import linear_model

import morphotrack.flow

/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
io_directory = '/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/morphotrack'

In [3]:
# get binary data for blood vesssel
vessel = io.imread(os.path.join(io_directory,'vessel_labkit.tif'))==1

In [4]:
# make guide vector from the binary image.
# load a mask of a white matter 
white_matter = io.imread(os.path.join(io_directory,'white_matter.tif'))!=0
# load a mask of a layer 1
layer1 = io.imread(os.path.join(io_directory,'l1.tif'))!=0

# make guide vector 
guide_vector = morphotrack.flow.guide_vector_generator_from_binaries(layer1, white_matter)

# or manually make a guide vector
# guide_coordinate1 = np.array([155.,122.,197.])
# guide_coordinate2 = np.array([110.,315.,213.])
# guide_vector = morphotrack.flow.guide_vector_generator(guide_coordinate1, guide_coordinate2)

In [5]:
vectors, img = morphotrack.flow.get_vectors_from_vessel(vessel, guide_vector, threshold=10, k=27, return_image=True)

start skeletonization
get vectors from skeletonized image
expand vectors to original image


In [ ]:
# visualize if needed
viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')

In [6]:
# Extract the vectors and positions using vessel.
vessel_position = np.array(np.where(vessel)).T

# The first smoothing
smoothed_vectors = morphotrack.flow.smooth_vectors(vessel_position,vectors)

(get_median_vector pid=580789) 
(get_median_vector pid=580693) 
(get_neighbor_vectors pid=580781) 
(get_neighbor_vectors pid=580693) 
(get_median_vector pid=580719) 
(get_neighbor_vectors pid=581248) 
(get_median_vector pid=580757) 
(get_neighbor_vectors pid=580719) 
(get_neighbor_vectors pid=580715) 
(get_median_vector pid=580757) 
(get_median_vector pid=580790) 
(get_median_vector pid=580765) 
(get_median_vector pid=580790) 
(get_neighbor_vectors pid=582369) 
(get_median_vector pid=580807) 
(get_neighbor_vectors pid=581247) 
(get_median_vector pid=580770) 
(get_median_vector pid=582531) 
(get_median_vector pid=580764) 
(get_neighbor_vectors pid=580756) 
(get_median_vector pid=580747) 
(get_neighbor_vectors pid=580781) 
(get_neighbor_vectors pid=580827) 
(get_neighbor_vectors pid=580727) 
(get_median_vector pid=581155) 
(get_median_vector pid=582803) 
(get_median_vector pid=580790) 
(get_median_vector pid=580727) 
(get_neighbor_vectors pid=580719) 
(get_median_vector pid=580718) 
(get

In [ ]:
# visualize if needed
viewer = napari.Viewer()

img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = smoothed_vectors

for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')

In [7]:
# remove sub-branches for the better fitting
keep = morphotrack.flow.remove_dissimilar_vectors(smoothed_vectors, vectors, threshold='otsu') # dots>=thresh

In [ ]:
# visualize if needed
viewer = napari.Viewer()

img = np.zeros(vessel.shape)
img[tuple(vessel_position[keep].T)] = 1
viewer.add_image(img, contrast_limits=[0,1], rgb=False, name='keep', colormap='green', blending='additive')
img = np.zeros(vessel.shape)
img[tuple(vessel_position[~keep].T)] = 1
viewer.add_image(img, contrast_limits=[0,1], rgb=False, name='keep', colormap='magenta', blending='additive')

In [8]:
# repeat the process to sophisticate the vector smoothing
vessel_position = np.array(np.where(vessel)).T[keep]

# The second smoothing
smoothed_vectors = morphotrack.flow.smooth_vectors(vessel_position,vectors[keep])

(get_neighbor_vectors pid=580739) 
(get_median_vector pid=589172) 
(get_median_vector pid=587642) 
(get_median_vector pid=589366) 
(get_neighbor_vectors pid=588174) 
(get_median_vector pid=588297) 
(get_median_vector pid=589638) 
(get_median_vector pid=588872) 
(get_median_vector pid=586038) 
(get_neighbor_vectors pid=587764) 
(get_neighbor_vectors pid=587073) 
(get_neighbor_vectors pid=587505) 
(get_neighbor_vectors pid=586038) 
(get_median_vector pid=587787) 
(get_neighbor_vectors pid=588503) 
(get_neighbor_vectors pid=589366) 
(get_median_vector pid=588969) 
(get_neighbor_vectors pid=587642) 
(get_median_vector pid=587073) 
(get_neighbor_vectors pid=589217) 
(get_median_vector pid=588684) 
(get_neighbor_vectors pid=589594) 
(get_neighbor_vectors pid=588392) 
(get_median_vector pid=585627) 
(get_neighbor_vectors pid=589172) 
(get_median_vector pid=589616) 
(get_neighbor_vectors pid=586584) 
(get_median_vector pid=588594) 
(get_median_vector pid=587340) 
(get_median_vector pid=587976)

In [9]:
# Fit to the nth polynomial
# degree = 5 # Overfitting may happen at the edge if degree is too high
# vec = smoothed_vectors
# poly = PolynomialFeatures(degree=degree)
# idx_ = poly.fit_transform(vessel_position)

# reg = linear_model.LinearRegression(fit_intercept=True)
# reg.fit(idx_,vec) # Fit the model
# reg.degree = degree # save information for polynomial degree for later use
import importlib
importlib.reload(morphotrack.flow)
reg = morphotrack.flow.polynomial_fitting(vessel_position, smoothed_vectors, degree=5)

In [10]:
# visualize if needed
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = smoothed_vectors

viewer = napari.Viewer()
for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='vessel', colormap=c, blending='additive')


pred = normalize(reg.predict(idx_), axis=1)
img = np.zeros(vessel.shape+(vessel.ndim,))
img[tuple(vessel_position.T)] = pred

for i, c in enumerate(['red','green','blue']):
    viewer.add_image(img[...,i], contrast_limits=[0,1], rgb=False, name='prediction', colormap=c, blending='additive')

In [ ]:
pickle.dump(reg, open(os.path.join(io_directory,'model_.pkl'), 'wb'))